<a href="https://colab.research.google.com/github/ng3339-prog/bootcamp_EDA/blob/main/Group10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Boostcamp Homework3
## This jupyter notebook is done by Weiyi Lu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path = '/content/drive/MyDrive/Data_Bootcamp/Homework/Homework3/data'

In [ ]:
app_data = pd.read_csv(path + '/application_data.csv')
prev_data = pd.read_csv(path + '/previous_application.csv')
columns_desc = pd.read_excel(path + '/columns_description.xlsx')

# Part1: analyze application_data.csv

## exploration

In [ ]:
app_data['TARGET'].value_counts(normalize=True)

In [ ]:
sns.countplot(x='TARGET', data=app_data)
plt.show()

Approximately 8% of customers experienced payment difficulties. Approximately 92% of customers made timely repayments.

In [ ]:
app_data.head()

In [ ]:
app_data[['TARGET', 'AMT_INCOME_TOTAL']].groupby('TARGET').median()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='TARGET', y='AMT_INCOME_TOTAL', data=app_data)
plt.ylim(0, 500000)  #limit the upper limit to prevent extreme values
plt.show()

In [ ]:
app_data[['TARGET', 'AMT_CREDIT']].groupby('TARGET').median()

In [ ]:
sns.boxplot(x='TARGET', y='AMT_CREDIT', data=app_data)
plt.show()

In [ ]:
app_data['CREDIT_INCOME_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_INCOME_TOTAL']
app_data[['TARGET', 'CREDIT_INCOME_RATIO']].groupby('TARGET').median()

In [ ]:
sns.boxplot(x='TARGET', y='CREDIT_INCOME_RATIO', data=app_data)
plt.show()

The graphs above show that Amount variables may not be the primary driver.

## Age

In [ ]:
app_data[['TARGET', 'DAYS_BIRTH']].groupby('TARGET').median()

In [ ]:
app_data['AGE_YEARS'] = app_data['DAYS_BIRTH'] / -365
plt.figure(figsize=(6,4))
sns.boxplot(x='TARGET', y='AGE_YEARS', data=app_data)
plt.show()


In [ ]:
app_data['AGE_GROUP'] = pd.cut(
    app_data['AGE_YEARS'],
    bins=[20,30,40,50,60,70],
    labels=['20-30','30-40','40-50','50-60','60-70']
)

age_default = app_data.groupby('AGE_GROUP')['TARGET'].mean()
age_default


In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x=age_default.index, y=age_default.values)

plt.ylabel("Default Rate")
plt.xlabel("Age Group")
plt.title("Default Rate by Age Group")
plt.ylim(0, 0.13)
plt.show()

Younger applicants (especially 20–30 age group) exhibit significantly higher default rates compared to older applicants. Default risk decreases consistently with age, indicating age is a strong predictor of repayment behavior.

## Years of employed

In [ ]:
app_data[['TARGET','DAYS_EMPLOYED']].groupby('TARGET').median()

In [ ]:
app_data['DAYS_EMPLOYED'].describe()


In [ ]:
(app_data['DAYS_EMPLOYED'] == 365243).sum()

In [ ]:
app_data['DAYS_EMPLOYED'] = app_data['DAYS_EMPLOYED'].replace(365243, np.nan)

In [ ]:
app_data['DAYS_EMPLOYED'].describe()


In [ ]:
app_data['EMP_YEARS'] = app_data['DAYS_EMPLOYED'] / -365

In [ ]:
app_data[['TARGET','EMP_YEARS']].groupby('TARGET').median()


In [ ]:
app_data['EMP_GROUP'] = pd.cut(
    app_data['EMP_YEARS'],
    bins=[0,2,5,10,20,40],
    labels=['0-2','2-5','5-10','10-20','20+']
)

emp_default = app_data.groupby('EMP_GROUP')['TARGET'].mean()
emp_default


In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x=emp_default.index, y=emp_default.values)

plt.ylabel("Default Rate")
plt.xlabel("Employment Years")
plt.title("Default Rate by Employment Length")
plt.ylim(0, 0.12)
plt.show()

Applicants with shorter employment history exhibit significantly higher default rates. Clients with less than 2 years of employment show a default rate of 11.2%, compared to only 4.26% among those with more than 20 years of employment. This indicates employment stability is a strong predictor of repayment behavior.

## Income type

In [ ]:
app_data['NAME_INCOME_TYPE'].value_counts()

Extreme small sample can be ignored

In [ ]:
income_type_default = app_data.groupby('NAME_INCOME_TYPE')['TARGET'].mean().sort_values(ascending=False)
income_type_default

In [ ]:
large_income = app_data['NAME_INCOME_TYPE'].value_counts()
large_income = large_income[large_income > 1000].index

income_plot = app_data[app_data['NAME_INCOME_TYPE'].isin(large_income)]

plt.figure(figsize=(8,4))
sns.barplot(x='NAME_INCOME_TYPE', y='TARGET', data=income_plot, estimator=np.mean)

plt.xticks(rotation=45)
plt.ylabel("Default Rate")
plt.title("Default Rate by Income Type")
plt.ylim(0, 0.12)
plt.show()

Among major income categories, applicants classified as “Working” exhibit the highest default rate (approximately 9.6%), followed by “Commercial associate” (7.5%), while “State servant” (5.8%) and “Pensioner” (5.4%) show significantly lower default rates. This pattern suggests that income stability plays an important role in repayment behavior, with more stable or institutional income sources associated with lower default risk.

## Education Level

In [ ]:
app_data['NAME_EDUCATION_TYPE'].value_counts()

In [ ]:
edu_default = app_data.groupby('NAME_EDUCATION_TYPE')['TARGET'].mean().sort_values(ascending=False)
edu_default

In [ ]:
plt.figure(figsize=(8,4))

sns.barplot(x='NAME_EDUCATION_TYPE', y='TARGET', data=app_data, estimator=np.mean)

plt.xticks(rotation=45)
plt.ylabel("Default Rate")
plt.title("Default Rate by Education Level")
plt.ylim(0, 0.12)
plt.show()

Education level is strongly associated with default risk. Applicants with lower education levels exhibit significantly higher default rates, while those with higher education demonstrate substantially lower risk. For instance, clients with only lower secondary education show a default rate of approximately 10.9%, compared to 5.4% among those with higher education and just 1.8% for academic degree holders. This suggests that educational attainment is a strong predictor of repayment behavior and may reflect differences in income stability and financial resilience.

##Family status

In [ ]:
family_count = app_data['NAME_FAMILY_STATUS'].value_counts()
family_count

In [ ]:
family_default = app_data.groupby('NAME_FAMILY_STATUS')['TARGET'].mean().sort_values(ascending=False)
family_default

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(x='NAME_FAMILY_STATUS', y='TARGET', data=app_data, estimator=np.mean)

plt.xticks(rotation=45)
plt.ylabel("Default Rate")
plt.title("Default Rate by Family Status")
plt.ylim(0, 0.11)
plt.show()

Family status is clearly associated with default risk. Applicants who are single or in civil marriages exhibit the highest default rates (around 9–10%), while married individuals show lower risk (7.6%), and widows demonstrate the lowest default rate (5.8%). This pattern suggests that greater household stability and social responsibility may be linked to stronger repayment behavior.

## Homeownership

In [ ]:
realty_count = app_data['FLAG_OWN_REALTY'].value_counts()
realty_count

In [ ]:
realty_default = app_data.groupby('FLAG_OWN_REALTY')['TARGET'].mean()
realty_default

In [ ]:
plt.figure(figsize=(6,4))

sns.barplot(x='FLAG_OWN_REALTY', y='TARGET', data=app_data, estimator=np.mean)

plt.ylabel("Default Rate")
plt.title("Default Rate by Real Estate Ownership")
plt.ylim(0, 0.09)
plt.show()

Homeownership is associated with lower default rates, but the effect is weak.

Compared to variables such as age, education, and years of work experience, homeownership is **not** a strong driving factor.

# Part2: analyze previous_application to extract "historical behavioral features".

In [ ]:
prev_data.head()

In [ ]:
prev_app_count = prev_data.groupby('SK_ID_CURR').size().reset_index(name='PREV_APP_COUNT')

prev_refused = prev_data[prev_data['NAME_CONTRACT_STATUS'] == 'Refused'].groupby('SK_ID_CURR').size().reset_index(name='PREV_REFUSED_COUNT')

prev_approved = prev_data[prev_data['NAME_CONTRACT_STATUS'] == 'Approved'].groupby('SK_ID_CURR').size().reset_index(name='PREV_APPROVED_COUNT')

In [ ]:
prev_features = prev_app_count.merge(prev_refused, on='SK_ID_CURR', how='left').merge(prev_approved, on='SK_ID_CURR', how='left')

prev_features = prev_features.fillna(0)

In [ ]:
prev_features['PREV_REFUSED_RATIO'] = (prev_features['PREV_REFUSED_COUNT'] / prev_features['PREV_APP_COUNT'])

In [ ]:
# Prevent exsiting data in previous merge
prev_cols = ['PREV_APP_COUNT', 'PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT', 'PREV_REFUSED_RATIO']
existing = [c for c in prev_cols if c in app_data.columns]

if existing:
    app_data = app_data.drop(columns=existing)

app_data = pd.merge(app_data, prev_features, on='SK_ID_CURR', how='left')

cols = ['PREV_APP_COUNT', 'PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT', 'PREV_REFUSED_RATIO']
app_data[cols] = app_data[cols].fillna(0)

In [ ]:
app_data['REFUSED_GROUP'] = pd.cut(
    app_data['PREV_REFUSED_RATIO'],
    bins=[0,0.2,0.5,0.8,1],
    labels=['0-20%','20-50%','50-80%','80-100%']
)

sns.barplot(x='REFUSED_GROUP', y='TARGET', data=app_data, estimator=np.mean)

plt.title("Default Rate by Previous Refusal Ratio")
plt.show()

Previous refusal history shows a strong and monotonic relationship with current default risk. As the ratio of previously refused applications increases, the default rate rises substantially—from about 7–8% for clients with less than 20% refusal history to nearly 18% for those with 80–100% refusal rates. This clear upward trend indicates that prior rejection behavior is a powerful predictor of future repayment difficulties and represents one of the strongest behavioral risk signals identified in the analysis.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

# 1. How much credit is granted vs how much is requested
plt.figure(figsize=(7,4))
sns.scatterplot(
    data=prev_data.sample(min(len(prev_data), 20000), random_state=42),
    x="AMT_APPLICATION",
    y="AMT_CREDIT",
    alpha=0.3
)
plt.title("Requested Loan Amount vs Granted Credit Amount")
plt.xlabel("Amount Requested")
plt.ylabel("Amount Granted")
plt.show()


# 2. Distribution of granted credit amounts
plt.figure(figsize=(7,4))
sns.histplot(prev_data["AMT_CREDIT"], bins=50, kde=True)
plt.title("Distribution of Granted Credit Amounts")
plt.xlabel("Amount Granted")
plt.show()


# 3. Distribution of requested loan amounts
plt.figure(figsize=(7,4))
sns.histplot(prev_data["AMT_APPLICATION"], bins=50, kde=True)
plt.title("Distribution of Requested Loan Amounts")
plt.xlabel("Amount Requested")
plt.show()


# 4. Loan term length vs granted credit
plt.figure(figsize=(7,4))
sns.scatterplot(
    data=prev_data.sample(min(len(prev_data), 20000), random_state=42),
    x="CNT_PAYMENT",
    y="AMT_CREDIT",
    alpha=0.3
)
plt.title("Loan Term Length vs Granted Credit")
plt.xlabel("Number of Payments")
plt.ylabel("Amount Granted")
plt.show()


# 5. Granted credit by application outcome
plt.figure(figsize=(8,4))
sns.boxplot(data=prev_data, x="NAME_CONTRACT_STATUS", y="AMT_CREDIT")
plt.title("Granted Credit Amount by Application Outcome")
plt.xlabel("Application Outcome")
plt.ylabel("Amount Granted")
plt.xticks(rotation=20)
plt.show()


# 6. Requested amount by contract type
plt.figure(figsize=(8,4))
sns.boxplot(data=prev_data, x="NAME_CONTRACT_TYPE", y="AMT_APPLICATION")
plt.title("Requested Loan Amount by Contract Type")
plt.xlabel("Contract Type")
plt.ylabel("Amount Requested")
plt.xticks(rotation=20)
plt.show()


# 7. Interest rates across pricing tiers
plt.figure(figsize=(8,4))
sns.boxplot(data=prev_data, x="NAME_YIELD_GROUP", y="RATE_INTEREST_PRIMARY")
plt.title("Interest Rates Across Yield Groups")
plt.xlabel("Yield Group")
plt.ylabel("Interest Rate")
plt.xticks(rotation=20)
plt.show()


# 8. Down payment rate vs granted credit
plt.figure(figsize=(7,4))
sns.scatterplot(
    data=prev_data.sample(min(len(prev_data), 20000), random_state=42),
    x="RATE_DOWN_PAYMENT",
    y="AMT_CREDIT",
    alpha=0.3
)
plt.title("Down Payment Rate vs Granted Credit")
plt.xlabel("Down Payment Rate")
plt.ylabel("Amount Granted")
plt.show()


# 9. Number of applications by day of the week
plt.figure(figsize=(8,4))
sns.countplot(data=prev_data, x="WEEKDAY_APPR_PROCESS_START")
plt.title("Number of Loan Applications by Day of Week")
plt.xlabel("Day of Week")
plt.ylabel("Application Count")
plt.xticks(rotation=20)
plt.show()


# 10. Approval rate by day of the week
prev_data["APPROVED"] = (prev_data["NAME_CONTRACT_STATUS"] == "Approved").astype(int)

approval_rate = (
    prev_data
    .groupby("WEEKDAY_APPR_PROCESS_START")["APPROVED"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(8,4))
sns.lineplot(data=approval_rate, x="WEEKDAY_APPR_PROCESS_START", y="APPROVED", marker="o")
plt.title("Loan Approval Rate by Day of Week")
plt.xlabel("Day of Week")
plt.ylabel("Approval Rate")
plt.ylim(0, 1)
plt.xticks(rotation=20)
plt.show()

NameError: name 'prev_data' is not defined

<Figure size 700x400 with 0 Axes>